In [1]:
import requests
import json
import shutil
from time import sleep
from PIL import Image
import io
import logging
import os
import shutil

In [3]:
logging.basicConfig(filename='downloader.log', encoding='utf-8', level=logging.WARNING)

In [4]:
url = 'https://api.scryfall.com/bulk-data'
bulks = requests.get(url)
bulk_dict = {s.get('type'): s for s in json.loads(bulks.text).get('data')}
default_url = bulk_dict.get('default_cards').get('download_uri')

In [5]:
default_cards=requests.get(default_url)
default_cards.ok

True

In [9]:
cards = json.loads(default_cards.text)
print(len(cards))

68516


In [ ]:
testcard = cards[42]

print(testcard.get('name'))

testcard.get()

In [7]:
target = '/mnt/d/cardimagescans'

if target:
    print('True')

True


In [10]:
def make_dir_dict(target):
    sets = os.listdir(target)
    card_dirs = {s : os.listdir(os.path.join(target, s)) for s in sets}
    return card_dirs

last_card = []
card_dirs = make_dir_dict(target)

for i, card in enumerate(cards):
    last_card = [i+1, card.get('name', False), card.get('set', False), card.get('collector_number', False), card.get('id', False)]
    assert not (False in last_card), 'False in last card: {}'.format(last_card)
    if not last_card[2] in card_dirs.keys(): # if cardset does not have a set folder...
        os.makedirs(os.path.join(target, last_card[2])) # make that set folder...
        card_dirs = make_dir_dict(target) # and refresh the directory dictionary
        if not '{}.png'.format(last_card[-1]) in card_dirs.get(last_card[2]): # check that card is not yet in set folder
            if card.get('highres_image'): # if card has highres png image available
                image_uris = card.get('image_uris', False)
                if (not image_uris == False) and (not image_uris == None):
                    png_uri = image_uris.get('png', False)
                    if png_uri:
                        image = requests.get(image_uris.get('png'))
                        sleep(0.1) # rate limiting, probably not necessary
                        if image.ok:
                            scryfall_id = last_card[-1]
                            Image.open(io.BytesIO(image.content)).save(os.path.join(target, last_card[2], '{}.png'.format(scryfall_id)))
                        else:
                            print('Card {} {} could not be retrieved. Error Code {}'. format(card.get('name'), card.get('set'), image.status_code))
                            logging.warning('{} {} {} not retreivable'.format(card.get('name'), card.get('set'), card.get('collector_number')))
                    else:
                        print('Card {} {} has no PNG uri. Skipping'.format(card.get('name'), card.get('set')))
                        logging.warning('{} {} {} no PNG uri'.format(card.get('name'), card.get('set'), card.get('collector_number')))
                else:
                    print('Card {} {} has no info on image uris. Skipping.'.format(card.get('name'), card.get('set')))
                    logging.warning('{} {} {} no image uris'.format(card.get('name'), card.get('set'), card.get('collector_number')))
            else:
                print('Card {} {} has no highres scan available. Skipping.'.format(card.get('name'), card.get('set')))
                logging.warning('{} {} {} no highres scan'.format(card.get('name'), card.get('set'), card.get('collector_number')))


In [ ]:
def make_dir_dict(target):
    sets = os.listdir(target)
    card_dirs = {s : os.listdir(os.path.join(target, s)) for s in sets}
    return card_dirs


#target = os.path.join('D:', 'cardimagescans')
last_card = []
card_dirs = make_dir_dict(target)

# TODO: implement new download algorithm that puts card images in respective set folders

for i, card in enumerate(cards):
    last_card = [i, card.get('name'), card.get('set'), card.get('collector_number')]
    if not last_card[2] in card_dirs.keys():
        os.makedirs(os.path.join(target, last_card[2]))
        card_dirs = make_dir_dict(target)  
        if not '{}.png'.format(card.get('id')) in card_dirs.get(last_card[2]):
            if card.get('highres_image'):
                image_uris = card.get('image_uris')
                if (not image_uris == False) and (not image_uris == None):
                    png_uri = image_uris.get('png')
                    if not png_uri == False:
                        image = requests.get(card.get('image_uris').get('png'))
                        sleep(0.1)
                        if image.ok:
                            scryfall_id=card.get('id')
                            Image.open(io.BytesIO(image.content)).save(os.path.join(target, last_card[2] '{}.png'.format(scryfall_id)))
                        else:
                            print('Card {} {} could not be retrieved. Error Code {}'. format(card.get('name'), card.get('set'), image.status_code))
                            logging.warning('{} {} {} not retreivable'.format(card.get('name'), card.get('set'), card.get('collector_number')))
                    else:
                        print('Card {} {} has no PNG uri. Skipping'.format(card.get('name'), card.get('set')))
                        logging.warning('{} {} {} no PNG uri'.format(card.get('name'), card.get('set'), card.get('collector_number')))
                else:
                    print('Card {} {} has no info on image uris. Skipping.'.format(card.get('name'), card.get('set')))
                    logging.warning('{} {} {} no image uris'.format(card.get('name'), card.get('set'), card.get('collector_number')))
            else:
                print('Card {} {} has no highres scan available. Skipping.'.format(card.get('name'), card.get('set')))
                logging.warning('{} {} {} no highres scan'.format(card.get('name'), card.get('set'), card.get('collector_number')))

In [ ]:
directory = os.listdir(target)
moved = 0
for i, card in enumerate(cards):
    id = card.get('id')
    s = card.get('set')
    file = '{}.png'.format(id)
    if file in directory:
        if not os.path.exists(os.path.join(target, s)):
            os.makedirs(os.path.join(target, s))
        shutil.move(os.path.join(target, file), os.path.join(target, s, file))
        moved +=1
    if moved % 200 == 0:
        print('{} cards moved into subfolders'.format(moved))
        print('{} card objects investigated so far'.format(i))

In [ ]:
png = requests.get(cards[42].get('image_uris').get('png'))
image =  Image.open(io.BytesIO(png.content))
display(image)

In [ ]:
cardnames = []
for i, card in enumerate(cards):
    cardnames.append([card.get('name'), i])
    image_uris = card.get('image_uris')
    png_uri = image_uris.get('png')
    

In [ ]:
problem_out = [0, 'Fury Sliver', 'tsp', '157', '0000579f-7b35-4ed3-b44c-db2a538066fe']
for value in problem_out:
    print(False == value)